# Dashboard Summarization 

## Importing Necessary Libraries

In [1]:
import ollama
import os
import requests
from bs4 import BeautifulSoup

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class website:

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        self.title = soup.title.string if soup.title else "No title found"
        self.text = soup.body.get_text(separator="\n", strip=True)


## Message Prompt 
The model like GPT4o, GPT families, qwen, llama families they expect to provide the two types of prompt which will make easier for the model to generate chat with user.

1. First one is, **System Prompt** This prompt means that you have to describe simply what you are doing, and you have to provide tone of the message that model should generate and in which method model should respond.
2. Second one is **User Prompt** This prompt means that you have to give the information directly to the model. 

In [3]:
system_prompt = "you are an assistant that analyzes the \
    the dashboard in website and provides a short executive summary for stockholders, CEOs, ignoring text that might be navigation related\
        response in markdown."

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nplease provide a detailed summary of the report for \
        The summary should be in a suitable form which could be sent through a mail for the executives\n"
    user_prompt += website.text
    return user_prompt

## Let's check 
Whether we were able to extract text form the website or not. 

In [4]:
ed = website("https://community.fabric.microsoft.com/t5/Data-Stories-Gallery/Revenue-amp-Profitability-Dashboard/td-p/4780272")
print(user_prompt_for(ed))

You are looking at a website titled 
	Revenue & Profitability Dashboard - Microsoft Fabric Community

please provide a detailed summary of the report for         The summary should be in a suitable form which could be sent through a mail for the executives
skip to main content
Join us at FabCon Vienna from September 15-18, 2025
The ultimate Fabric, Power BI, SQL, and AI community-led learning event. Save €200 with code FABCOMM.
Get registered
Skip to main content
Fabric Community
Forums
Power BI forums
Data Engineering forums
Data Warehouse forums
Data Science forums
Data Factory forums
Real-Time Intelligence forums
Databases forums
Fabric platform forums
Inspiration
Contests
Power BI galleries
Data Engineering galleries
Data Warehouse galleries
Data Science galleries
Data Factory galleries
Real-Time Intelligence galleries
Databases galleries
Fabric platform galleries
Ideas
Submit ideas & vote
Fabric roadmap
Communities
Join a user group
Find an event
Become a Super User
Azure Data Com

In [5]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt}, 
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [6]:
print(messages_for(ed))

[{'role': 'system', 'content': 'you are an assistant that analyzes the     the dashboard in website and provides a short executive summary for stockholders, CEOs, ignoring text that might be navigation related        response in markdown.'}, {'role': 'user', 'content': 'You are looking at a website titled \n\tRevenue & Profitability Dashboard - Microsoft Fabric Community\n\nplease provide a detailed summary of the report for         The summary should be in a suitable form which could be sent through a mail for the executives\nskip to main content\nJoin us at FabCon Vienna from September 15-18, 2025\nThe ultimate Fabric, Power BI, SQL, and AI community-led learning event. Save €200 with code FABCOMM.\nGet registered\nSkip to main content\nFabric Community\nForums\nPower BI forums\nData Engineering forums\nData Warehouse forums\nData Science forums\nData Factory forums\nReal-Time Intelligence forums\nDatabases forums\nFabric platform forums\nInspiration\nContests\nPower BI galleries\nDa

## Chatting with Ollama 

I have already told you how to download and install different types of models throug Ollama and it will be free/not paid but it does not have accuracy like GPT families. but this type of LLM can be used locally. 

In [7]:
def summarize(url, website):
    website = website(url)
    response = ollama.chat(
        model="qwen3", 
        messages=messages_for(website)
    )

    return response['message']["content"]

url = "https://community.fabric.microsoft.com/t5/Data-Stories-Gallery/Revenue-amp-Profitability-Dashboard/td-p/4780272"
print(summarize("https://community.fabric.microsoft.com/t5/Data-Stories-Gallery/Revenue-amp-Profitability-Dashboard/td-p/4780272", website))

<think>
Okay, let's tackle this query. The user wants an executive summary of the Revenue & Profitability Dashboard from Microsoft Fabric Community. They mentioned sending it to stockholders and CEOs, so the summary needs to be concise and highlight key metrics without the navigation text.

First, I need to parse the provided content. The main dashboard info is in the initial post by Arbaz_Ahmad. The key points are Year-over-Year analysis of Revenue, Profit, Unit Sales, segmented by region, product, and customer group. The dashboard features include a clean layout, contextual metrics, clear comparisons, and insights at product and region levels. There's also a mention of the dashboard being a strategic tool for decision-making.

I should skip any navigation-related text like "Skip to main content" or event registration links. The user wants the summary in markdown, suitable for an email. So I'll structure it with a subject line, greeting, key metrics, insights, and a closing. Make sure

# Executive Summary: Revenue & Profitability Dashboard  
**Date:** July 29, 2025  

---

## **Key Insights**  
This dashboard provides a **Year-over-Year (YoY)** analysis of core financial metrics, including **Revenue, Profit, and Unit Sales**, segmented by **region, product, and customer group**. The design emphasizes clarity, context, and actionable insights to support strategic decision-making.  

---

### **Core Performance Metrics**  
1. **Revenue**  
   - **Global Revenue Growth:** [Insert % growth] YoY, driven by [key regions/products].  
   - **Regional Performance:** Highlighted growth in [Region A] and [Region B], with [Region C] showing decline.  
   - **Product Segmentation:** [Product X] led revenue growth, while [Product Y] faced margin pressures.  

2. **Profitability**  
   - **Net Profit Margin:** [Insert %] YoY, influenced by [cost factors, pricing strategies].  
   - **Customer Group Analysis:** [Group A] contributed [X]% to profits, while [Group B] showed declining margins.  

3. **Unit Sales**  
   - **Volume Trends:** [X]% increase in unit sales for [Product Z], offset by declines in [Product W].  
   - **Regional Distribution:** [Region D] saw the highest unit growth, while [Region E] experienced inventory challenges.  

---

### **Strategic Highlights**  
- **Regional Insights:** [Region F] emerged as a high-growth market, warranting increased investment.  
- **Product Opportunities:** [Product G] demonstrated strong customer adoption, suggesting potential for expansion.  
- **Customer Segmentation:** [Group C] showed higher lifetime value, indicating focus areas for retention strategies.  

---

### **Dashboard Design & Utility**  
- **Clean Layout:** Reduces cognitive load and prioritizes critical metrics.  
- **Contextual Alignment:** Metrics are tied to business goals (e.g., profitability targets, regional expansion plans).  
- **Decision Support:** Clear YoY comparisons and drill-down capabilities enable rapid, data-driven decisions.  

---

## **Next Steps**  
- Monitor [Region H] for potential revenue risks.  
- Explore opportunities to scale [Product I] in high-growth markets.  
- Reassess pricing strategies for [Product J] to improve margins.  

---

**Follow Arbaz Ahmad for updates:** [LinkedIn Profile](https://www.linkedin.com/in/arbaz-ahmad-/)  
**Dashboard Link:** [Revenue & Profitability Dashboard](https://www.microsoft.com/en-us/fabric/community)  


